In [1]:
from pycco.parsers import *

In [2]:
params_list([tokenize('int sum(void) { return a; }')[3]])

ParserResult(position=Position(index=0), result=[], description=(type + optional '*' + identifier sep_by Token(kind=<TokenKind.SYMBOL: 4>, value=',', start=-1, end=-1)), error=None)

In [3]:
function_args(tokenize('''
(void) { return a; }
''')).result

Parser(Token(kind=<TokenKind.TYPE: 3>, value='void', start=-1, end=-1))

In [3]:
%pdb

Automatic pdb calling has been turned ON


In [4]:
print(parse_c_code('''
//yolo
int sum() { return a; }
'''))


int sum(void)
{

	return a;
}




In [9]:
    source = "int main() { return 0; }"
    ast = parse_c_code(source)

    assert isinstance(ast, Code)
    assert len(ast.statements) == 1
    func = ast.statements[0]
    assert isinstance(func, Function)

    # Check function's "var" (return type + function name)
    assert isinstance(func.var, VariableDecl)
    assert func.var.type.name == "int"
    assert isinstance(func.var.name, Ident)
    assert func.var.name.name == "main"

    # Check args
    assert len(func.args) == 0

    # Check body
    assert len(func.body) == 0  # Contains the return statement
    assert isinstance(func.ret, Return)
    assert isinstance(func.ret.value, Number)
    assert func.ret.value.value == "0"

In [6]:
func.body

[]

In [3]:
parse_c_code('(int x)')

ValueError: too many values to unpack (expected 2)

In [5]:
tokens

[Token(kind=<TokenKind.TYPE: 3>, value='int', start=34, end=36),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='x', start=38, end=38),
 Token(kind=<TokenKind.SYMBOL: 4>, value=';', start=39, end=39),
 Token(kind=<TokenKind.EOF: 10>, value=None, start=41, end=41)]

In [5]:
eof_parser('')

ParserResult(position=Position(index=0), result=None, description=None, error=ParseError at index 0: expected EOF, found 't')

In [11]:
[*'jkl']

['j', 'k', 'l']

In [10]:
tokenize('*** ')

[Token(kind=<TokenKind.OPERATOR: 5>, value='*', start=0, end=0),
 Token(kind=<TokenKind.OPERATOR: 5>, value='*', start=1, end=1),
 Token(kind=<TokenKind.OPERATOR: 5>, value='*', start=2, end=2),
 Token(kind=<TokenKind.EOF: 10>, value=None, start=4, end=4)]

In [7]:
tokens

[Token(kind=<TokenKind.TYPE: 3>, value='float', start=0, end=4),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='pi', start=6, end=7),
 Token(kind=<TokenKind.OPERATOR: 5>, value='=', start=9, end=9),
 Token(kind=<TokenKind.NUMBER: 7>, value='3.14', start=11, end=14),
 Token(kind=<TokenKind.SYMBOL: 4>, value=';', start=15, end=15),
 Token(kind=<TokenKind.OPERATOR: 5>, value='/', start=17, end=17),
 Token(kind=<TokenKind.OPERATOR: 5>, value='/', start=18, end=18),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='Initialize', start=20, end=29),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='pi', start=31, end=32),
 Token(kind=<TokenKind.EOF: 10>, value=None, start=34, end=34)]

In [6]:
    parser = match("a") | match("b")
    stream = "c"
    position = Position(0)
    result = parser(stream, position)

[DEBUG] Calling parser '('a' | 'b')' at Position(index=0)
[DEBUG] Parser '('a' | 'b')' failed at Position(index=0) with error: ParseError at index 0: expected 'b', found 'c'


In [5]:
parser.description

"('a' | 'b')"

In [74]:
(match('5')+match(' ').optional()).sep_by(',')('5, 5')

ParserResult(index=1, result=[['5']], description=(("5" + " "?) sep_by ","))

In [77]:
args(tokenize('(int x  int y)')).result

In [70]:
args(tokenize('(void)'))
# ParserResult(index=7, result=[VariableDecl(type=Type(name='void', pointer=False), name=Ident(name='x'), semicolon=False), VariableDecl(type=Type(name='int', pointer=False), name=Ident(name='y'), semicolon=False)], description=(("(" >> (((("TYPE" + "*"?) + "IDENTIFIER") sep_by ",") | "void")) << ")"))

ParserResult(index=-1, result=None, description=(("(" >> (((("TYPE" + "*"?) + "IDENTIFIER") sep_by ",") | fn`<lambda>`)) << ")"))

In [44]:
(match(void) | variable.sep_by(Token(TokenKind.SYMBOL, ",")))(tokenize())

ParserResult(index=1, result=void, description=("void" | ((("TYPE" + "*"?) + "IDENTIFIER") sep_by ",")))

In [49]:
(variable<<Token(TokenKind.SYMBOL, ",")).until(variable)(tokenize('void x, int y')).result

[VariableDecl(type=Type(name='void', pointer=False), name=Ident(name='x'), semicolon=False),
 Token(kind=<TokenKind.SYMBOL: 4>, value=',', start=7, end=7)]

In [36]:
match(void)(tokenize('void'))

Token(kind=<TokenKind.TYPE: 3>, value='void', start=4, end=7)

In [24]:
code='''
int main(void) {
    // test case w/ multi-digit constant
    return 100;
}
'''

In [25]:
tokenize(code)

[Token(kind=<TokenKind.TYPE: 3>, value='int', start=4, end=6),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='main', start=9, end=12),
 Token(kind=<TokenKind.SYMBOL: 4>, value='(', start=10, end=10),
 Token(kind=<TokenKind.TYPE: 3>, value='void', start=14, end=17),
 Token(kind=<TokenKind.SYMBOL: 4>, value=')', start=15, end=15),
 Token(kind=<TokenKind.SYMBOL: 4>, value='{', start=17, end=17),
 Token(kind=<TokenKind.KEYWORD: 2>, value='return', start=69, end=74),
 Token(kind=<TokenKind.NUMBER: 7>, value='100', start=73, end=75),
 Token(kind=<TokenKind.SYMBOL: 4>, value=';', start=74, end=74),
 Token(kind=<TokenKind.SYMBOL: 4>, value='}', start=76, end=76),
 Token(kind=<TokenKind.EOF: 10>, value=None, start=77, end=77)]

In [26]:
function(tokenize(code))

ParserResult(index=-1, result=None, description=(((("TYPE" + "*"?) + "IDENTIFIER") + (("(" >> (((("TYPE" + "*"?) + "IDENTIFIER") sep_by ",") | "void")) << ")")) + (("{" >> (expression* + (("return" >> expression) << ";")?)) << "}")))

In [10]:
ast = parse_c_code('''
int main(void){
    int x;
    int y;
    y = 5;
    x=(1+ y);
    return 1;
}''')

In [11]:
from pprint import pprint

In [12]:
pprint(ast)

Code(statements=[Function(var=VariableDecl(type=Type(name='int', pointer=False),
                                           name=Ident(name='main'),
                                           semicolon=False),
                          args=[],
                          body=[VariableDecl(type=Type(name='int',
                                                       pointer=False),
                                             name=Ident(name='x'),
                                             semicolon=True),
                                VariableDecl(type=Type(name='int',
                                                       pointer=False),
                                             name=Ident(name='y'),
                                             semicolon=True),
                                Assign(var=Ident(name='y'),
                                       to=Number(value='5',
                                                 type=Type(name='int',
                              

In [13]:
print(ast.compile('arm'))

.section .text
.global main
main:
    PUSH {LR}    @ Save link register
    @ var decl: x
    @ var decl: y
@ Assignment
    MOV R0, #5    @ load immediate 5
    STR R0, [R1]    @ store into y
@ Assignment
    MOV R0, #1    @ load immediate 1
    PUSH {R0}
    LDR R0, =y    @ Ident y
    POP {R1}
    ADD R0, R1, R0    @ R0 = R1 + R0
    STR R0, [R1]    @ store into x
    MOV R0, #1    @ load immediate 1
    MOV PC, LR    @ Return from function


In [37]:
(expression).until(TokenKind.EOF)(tokenize('1-x')).result

[Token(kind=<TokenKind.NUMBER: 7>, value='1', start=1, end=1),
 Token(kind=<TokenKind.OPERATOR: 5>, value='-', start=2, end=2),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='x', start=3, end=3)]

In [47]:
parse_tokens(tokenize('x+1 ')).result

[Ident(name='x'),
 UnaryOp(operator='+', operand=Number(value='1', type=Type(name='int', pointer=False)))]

In [6]:
# group parens tokens
# recurse
# no parens -> precedence of binops
    # split on highest precendence binop that occurs first
    # continue while some binop token exists
    # if no binop token exists -> parse with all expressions not including binop (or temp define binop to be something else)

In [7]:
%pdb

Automatic pdb calling has been turned OFF


In [24]:
parenthesized.describe('parenthesis')

Parser(parenthesis)

In [25]:
%pdb

Automatic pdb calling has been turned OFF


In [28]:
parenthesized(tokenize('(x+1)'))

ParserResult(index=5, result=[Ident(name='x')], description=parenthesis)

In [ ]:
@parser
def binop(stream, index: int):
    left

In [3]:
non_binop_expression = number | string | ident | paren_expr

In [2]:
# 1) Our base expression (numbers, strings, ident)

base_expr = (number | string | ident | paren_expr).describe("base_expr")


# 2) The binop chain
def map_binops(parsed_list: List) -> ast.Expression:
    """
    parsed_list = [left_expr, [ (op, right_expr), (op, right_expr), ... ]]
    """
    left = parsed_list[0]
    pairs = parsed_list[1]
    for (op, right_expr) in pairs:
        left = ast.BinaryOp(left, op.value, right_expr)
    return left

binop = (
    base_expr + 
    ( any_of_enum(BinaryOperator) + base_expr ).many()
) @ map_binops

# 3) The unary_op (already defined)
def map_unary_op(elements: List[Token | ast.Expression]) -> ast.UnaryOp:
    op, operand = elements
    return ast.UnaryOp(operator=op.value, operand=operand)

unary_op = (
    any_of_enum(UnaryOperator) + expression
) @ map_unary_op

# 4) Final expression definition
expression.define(
    unary_op | binop
)

# Now 'expression' references a parser that
# - tries unary_op first, or
# - tries binop (chain of base_expr with binary operators).


Parser((((one of UnaryOperator members + None) @ map_unary_op) | ((base_expr + (one of BinaryOperator members + base_expr)*) @ map_binops)))

In [8]:
.result

RecursionError: maximum recursion depth exceeded

In [13]:
# Primary Expressions: Numbers, Strings, Identifiers, and Grouped Expressions
@parser
def primary_expression(stream: List[Token], index: int) -> ParserResult[ast.Expression]:
    return (
        number
        | string
        | ident
        | (open_paren >> expression << close_paren)  # Grouped expressions: (x + y)
    ).parse_fn(stream, index)

# Unary Operations
unary_op = (any_of_enum(UnaryOperator) + primary_expression) @ map_unary_op

# Binary Operations (Left-Associative with Precedence)
def map_binop(elements: List[ast.Node | Token]) -> ast.BinaryOp:
    left, op, right = elements
    return ast.BinaryOp(left=left, operator=op.value, right=right)

binary_operator = any_of_enum(BinaryOperator)

@parser
def binop(stream: List[Token], index: int) -> ParserResult[ast.Expression]:
    # Start by parsing the left operand (unary or primary expression)
    import pdb; pdb.set_trace()
    left_result = unary_op.parse_fn(stream, index)
    current_index = left_result.index  # Correct attribute
    left = left_result.result

    while True:
        try:
            # Parse the operator
            op_result = binary_operator.parse_fn(stream, current_index)
            operator = op_result.result
            current_index = op_result.index  # Correct attribute

            # Parse the right-hand operand
            right_result = unary_op.parse_fn(stream, current_index)
            right = right_result.result
            current_index = right_result.index  # Correct attribute

            # Combine into a BinaryOp
            left = ast.BinaryOp(left=left, operator=operator.value, right=right)

        except Exception:
            # If parsing fails, break the loop
            break

    return ParserResult(index=current_index, result=left)


# Function Calls
def map_function_call(elements: List[ast.Node | Token]) -> ast.FunctionCall:
    name, args = elements
    return ast.FunctionCall(name=name, args=args)

function_call = (
    ident
    + open_paren
    >> expression.sep_by(comma)
    << close_paren
) @ map_function_call

# Array Indexing
def map_array_index(elements: List[ast.Node]) -> ast.ArrayIndex:
    array, index = elements
    return ast.ArrayIndex(array=array, index=index)

array_index = ident + open_bracket >> expression << close_bracket @ map_array_index

# Struct Access
def map_struct_access(elements: List[ast.Node | Token]) -> ast.StructAccess:
    obj, op, field = elements
    return ast.StructAccess(obj=obj, operator=op.value, field=field)

struct_access = (
    ident
    + any_of(Symbol.DOT, Symbol.ARROW)
    + ident
) @ map_struct_access

# Comprehensive Expression Parser
@parser
def expression(stream: List[Token], index: int) -> ParserResult[ast.Expression]:
    return (
        struct_access
        | array_index
        | function_call
        | binop
        | unary_op
        | primary_expression
    ).parse_fn(stream, index)


In [14]:
# If-Else Parser
def map_if_else(elements: List[Token | ast.Node]) -> ast.IfStatement:
    condition, then_body, else_body = elements
    return ast.IfStatement(condition, then_body, else_body)

if_else = (
    match(Token(TokenKind.KEYWORD, "if"))
    >> open_paren
    >> expression
    << close_paren
    + open_bracket
    >> statement.many()
    << close_bracket
    + (match(Token(TokenKind.KEYWORD, "else")) >> open_bracket >> statement.many() << close_bracket).optional()
) @ map_if_else

In [37]:
expression(tokenize('x>0 ')).result

> /var/folders/vl/0mv20zzj0ld26z8h0ngg3wn00000gn/T/ipykernel_38709/2568700022.py(19)map_binop()
     17 
     18 def map_binop(elems):
---> 19     import pdb; pdb.set_trace()
     20     left, op, right = elems
     21     return ast.BinaryOp(left=left, operator=op.value, right=right)



ipdb>  elems


[Ident(name='x')]


ipdb>  q


In [19]:
(match(Token(TokenKind.KEYWORD, "if"))
    >> open_paren
>> expression)(tokenize('''
if (x > 0) {
    y = 1;
} else {
    y = -1;
}
'''))

RecursionError: maximum recursion depth exceeded

In [ ]:
# While Loop Parser
def map_while(elements: List[Token | ast.Node]) -> ast.WhileLoop:
    condition, body = elements
    return ast.WhileLoop(condition, body)

while_loop = (
    match(Token(TokenKind.KEYWORD, "while"))
    >> open_paren
    >> expression
    << close_paren
    + open_bracket
    >> statement.many()
    << close_bracket
) @ map_while

# For Loop Parser
def map_for(elements: List[Token | ast.Node]) -> ast.For:
    init, condition, increment, body = elements
    return ast.For(init, condition, increment, body)

for_loop = (
    match(Token(TokenKind.KEYWORD, "for"))
    >> open_paren
    >> variable_assign
    << semicolon
    + expression
    << semicolon
    + variable_assign
    << close_paren
    + open_bracket
    >> statement.many()
    << close_bracket
) @ map_for

# Function Call Parser
def map_function_call(elements: List[Token | ast.Node]) -> ast.FunctionCall:
    name, args = elements
    return ast.FunctionCall(name, args)

function_call = (
    ident
    + open_paren
    >> expression.sep_by(comma)
    << close_paren
) @ map_function_call

# Array Index Parser
def map_array_index(elements: List[Token | ast.Node]) -> ast.ArrayIndex:
    array, index = elements
    return ast.ArrayIndex(array=array, index=index)

array_index = ident + open_bracket >> expression << close_bracket @ map_array_index

# Struct Access Parser
def map_struct_access(elements: List[Token | ast.Node]) -> ast.StructAccess:
    obj, operator, field = elements
    return ast.StructAccess(obj=obj, operator=operator.value, field=field)

struct_access = (
    ident
    + any_of_enum(OtherOperator)
    .filter(lambda op: op in {OtherOperator.DOT, OtherOperator.ARROW})
    + ident
) @ map_struct_access

# Update Statement Parser
@parser
def statement(stream: List[Token], index: int) -> ParserResult[ast.Statement]:
    return (
        if_else | while_loop | for_loop | variable_decl | variable_assign | return_
    ).parse_fn(stream, index)

# Update Expression Parser
@parser
def expression(stream: List[Token], index: int) -> ParserResult[ast.Expression]:
    return (
        struct_access | array_index | function_call | unary_op | binop | ident | number | string
    ).parse_fn(stream, index)

AttributeError: module 'pycco.ast' has no attribute 'For'

In [48]:
binop(tokenize('2+"2" ')).result

BinaryOp(left=Number(value='2', type=Type(name='int', pointer=False)), operator='+', right=StringLiteral(value='2'))

In [30]:
close_bracket

Parser("}")

In [31]:
statement.many(1)(tokenize('return 2;')).result

In [32]:
(statement.until(close_bracket)@map_return)(tokenize('return 2;}')).result

Return(value=[Token(kind=<TokenKind.KEYWORD: 2>, value='return', start=6, end=11), Token(kind=<TokenKind.NUMBER: 7>, value='2', start=8, end=8), Token(kind=<TokenKind.SYMBOL: 4>, value=';', start=9, end=9)])

In [33]:
statement(tokenize('return 2;'))

ParserResult(index=-1, result=None, description=None)

In [34]:
r=function(tokenize('''
int main(char* x){
int *x=
3;
    return 2;
}''')).result

In [35]:
r.ret

Return(value=Number(value='2', type=Type(name='int', pointer=False)))

In [36]:
print(r)


int main(char* x)
{
	int* x = 3;
	return 2;
}




In [19]:
r=(((name@map_ident | variable_@map_variable_decl) << Token(TokenKind.OPERATOR, '='))+expression<<semicolon)(tokenize('int x=3;')).result

In [20]:
map_variable_assign(r)

Assign(var=VariableDecl(type=Type(name='int', pointer=False), name=Ident(name='x'), semicolon=False), to=Number(value='3', type=Type(name='int', pointer=False)))

In [ ]:
# Binary Expression
def map_binary_expression(tokens: List[Token]) -> ast.BinaryExpression:
    """
    Map tokens to a BinaryExpression.
    Assumes tokens are in the order: left operator right
    """
    left = tokens[0]
    operator = tokens[1].value
    right = tokens[2]
    return ast.BinaryExpression(
        left=ast.Ident(left.value) if left.kind == TokenKind.IDENTIFIER else left,
        operator=operator,
        right=ast.Ident(right.value) if right.kind == TokenKind.IDENTIFIER else right,
    )

expression = Parser()  # Forward declaration for expressions
binary_expression = (expression + TokenKind.OPERATOR + expression) @ map_binary_expression

# If Statement
def map_if_statement(tokens: List[Token]) -> ast.IfStatement:
    """
    Map tokens to an IfStatement.
    Assumes tokens are in the order: if (condition) { then_branch } [else { else_branch }]
    """
    condition = tokens[0]  # First token after `if` and `(`
    then_branch = tokens[1]  # Tokens within the `{}` following the condition
    else_branch = tokens[2] if len(tokens) > 2 else None  # Optional `else` branch
    return ast.IfStatement(condition=condition, then_branch=then_branch, else_branch=else_branch)

if_statement = (
    match(Token(TokenKind.KEYWORD, "if"))
    >> open_paren
    >> expression
    << close_paren
    + open_bracket
    >> variable_decl.until(close_bracket)
    << close_bracket
    + (
        match(Token(TokenKind.KEYWORD, "else"))
        >> open_bracket
        >> variable_decl.until(close_bracket)
        << close_bracket
    ).optional()
) @ map_if_statement

# While Loop
def map_while_loop(tokens: List[Token]) -> ast.WhileLoop:
    """
    Map tokens to a WhileLoop.
    Assumes tokens are in the order: while (condition) { body }
    """
    condition = tokens[0]  # First token after `while` and `(`
    body = tokens[1]  # Tokens within the `{}` following the condition
    return ast.WhileLoop(condition=condition, body=body)

while_loop = (
    match(Token(TokenKind.KEYWORD, "while"))
    >> open_paren
    >> expression
    << close_paren
    + open_bracket
    >> variable_decl.until(close_bracket)
    << close_bracket
) @ map_while_loop

# Function
def map_function(tokens: List[Token]) -> ast.Function:
    """
    Map tokens to a Function.
    Assumes tokens are in the order: return_type name(args) { body }
    """
    return_type = ast.Type(tokens[0].value)
    name = ast.Ident(tokens[1].value)
    args = [map_variable_decl(arg) for arg in tokens[2]]  # Tokens within `()`
    body = tokens[3]  # Tokens within `{}`
    return ast.Function(
        var=ast.VariableDecl(type=return_type, name=name),
        args=args,
        body=body,
    )

# Forward declaration for statement
statement = Parser()

# Function parser
def map_function(tokens: List[Token]) -> ast.Function:
    """
    Map tokens to a Function.
    Assumes tokens are in the order: return_type name(args) { body }
    """
    return_type = ast.Type(tokens[0].value)
    name = ast.Ident(tokens[1].value)
    args = [map_variable_decl(arg) for arg in tokens[2]]  # Tokens within `()`
    body = tokens[3]  # Tokens within `{}`
    return ast.Function(
        var=ast.VariableDecl(type=return_type, name=name),
        args=args,
        body=body,
    )

function = (
    type
    + name
    + (open_paren >> variable.until(comma.optional() + close_paren))
    << close_paren
    + open_bracket
    >> statement.until(close_bracket)
    << close_bracket
) @ map_function

# Define the statement parser
statement.define(variable_decl | if_statement | while_loop | binary_expression)


# Update the `_expression` placeholder to include binary_expression
expression.define(binary_expression | variable)


# Entry Point for Parsing a Program
program = function.until(TokenKind.EOF)

In [59]:
source_code = """
int* main(args) {
    int* x = 42;

    return 0;
}
"""

In [60]:
tokens = tokenize(source_code)

In [61]:
function(tokens)

ParserResult(index=-1, result=None, description=(((((("TYPE" + "*"?) + "IDENTIFIER") @ map_variable_decl) + ((("(" >> (((("TYPE" + "*"?) + "IDENTIFIER") @ map_variable_decl) sep_by ",")) << ")") @ <lambda>)) + (("{" >> (None* + ((("return" >> None) << ";") @ map_return)?)) << "}")) @ map_function))

In [12]:
variable = type + star.optional() + name
variable_decl = (variable + semicolon.optional()) @ map_variable_decl

In [13]:
variable_decl(tokenize('int main'))

AttributeError: 'str' object has no attribute 'value'

In [7]:
variable_decl(tokenize('int main'))

ParserResult(index=2, result=[Token(kind=<TokenKind.TYPE: 3>, value='int', start=3, end=5), Token(kind=<TokenKind.IDENTIFIER: 1>, value='main', start=8, end=11)], description=((("TYPE" + "*"?) + "IDENTIFIER") + ";"?))

In [9]:
parse(tokens)

ParseError: Parsing failed due to an unexpected error.
At token index 0:
int main ( args )
^
Token causing issue: int (kind: TYPE)

In [3]:
r=(variable@variable_map)(tokenize('int *main'))

NameError: name 'variable_map' is not defined

In [6]:
r.result.tokens

[Token(kind=<TokenKind.TYPE: 3>, value='int', start=3, end=5),
 Token(kind=<TokenKind.OPERATOR: 5>, value='*', start=5, end=5),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='main', start=9, end=12)]

In [4]:
(type + name)(tokenizer(('int main')))

ParserResult(index=-1, result=None, description=("TYPE" + "IDENTIFIER"))

In [6]:
(type + name + (open_paren >> variable.until(comma.optional() + close_paren)))(source_code)

ParserResult(index=-1, result=None, description=(("TYPE" + "IDENTIFIER") + ("(" >> ((("TYPE" + "*"?) + "IDENTIFIER") until (","? + ")")))))

In [3]:
type(tokenize('int'))

ParserResult(index=1, result=int, description="TYPE")

In [4]:
variable_decl(tokenize('int* x;'))

ParserResult(index=4, result=[Token(kind=<TokenKind.TYPE: 3>, value='int', start=3, end=5), Token(kind=<TokenKind.OPERATOR: 5>, value='*', start=4, end=4), Token(kind=<TokenKind.IDENTIFIER: 1>, value='x', start=6, end=6), Token(kind=<TokenKind.SYMBOL: 4>, value=';', start=7, end=7)], description=((("TYPE" + "*"?) + "IDENTIFIER") + ";"))

In [9]:
function(tokenize(source_code))

ParserResult(index=21, result=[Token(kind=<TokenKind.TYPE: 3>, value='int', start=25, end=27), Token(kind=<TokenKind.OPERATOR: 5>, value='*', start=26, end=26), Token(kind=<TokenKind.IDENTIFIER: 1>, value='x', start=28, end=28), Token(kind=<TokenKind.OPERATOR: 5>, value='=', start=30, end=30), Token(kind=<TokenKind.NUMBER: 7>, value='42', start=33, end=34), Token(kind=<TokenKind.SYMBOL: 4>, value=';', start=34, end=34), Token(kind=<TokenKind.IDENTIFIER: 1>, value='printf', start=85, end=90), Token(kind=<TokenKind.SYMBOL: 4>, value='(', start=86, end=86), Token(kind=<TokenKind.STRING_LITERAL: 6>, value='Hello, World!', start=101, end=113), Token(kind=<TokenKind.SYMBOL: 4>, value=')', start=102, end=102), Token(kind=<TokenKind.SYMBOL: 4>, value=';', start=103, end=103), Token(kind=<TokenKind.KEYWORD: 2>, value='return', start=114, end=119), Token(kind=<TokenKind.NUMBER: 7>, value='0', start=116, end=116), Token(kind=<TokenKind.SYMBOL: 4>, value=';', start=117, end=117)], description=((((

In [10]:
tokenize(source_code)

[Token(kind=<TokenKind.TYPE: 3>, value='int', start=4, end=6),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='main', start=9, end=12),
 Token(kind=<TokenKind.SYMBOL: 4>, value='(', start=10, end=10),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='args', start=14, end=17),
 Token(kind=<TokenKind.SYMBOL: 4>, value=')', start=15, end=15),
 Token(kind=<TokenKind.SYMBOL: 4>, value='{', start=17, end=17),
 Token(kind=<TokenKind.TYPE: 3>, value='int', start=25, end=27),
 Token(kind=<TokenKind.OPERATOR: 5>, value='*', start=26, end=26),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='x', start=28, end=28),
 Token(kind=<TokenKind.OPERATOR: 5>, value='=', start=30, end=30),
 Token(kind=<TokenKind.NUMBER: 7>, value='42', start=33, end=34),
 Token(kind=<TokenKind.SYMBOL: 4>, value=';', start=34, end=34),
 Token(kind=<TokenKind.IDENTIFIER: 1>, value='printf', start=85, end=90),
 Token(kind=<TokenKind.SYMBOL: 4>, value='(', start=86, end=86),
 Token(kind=<TokenKind.STRING_LITERAL: 6>, value='Hello, Worl

In [11]:
bad_code = """
int main() {
    int x = 42;
    /*
    multiline comment
    */
    printf(Hello, World!");
    return 0;
}
"""

In [12]:
tokenize(bad_code)

ValueError: Tokenization Error:
Unexpected token at line 7, column 25:

    multiline comment
    */
    printf(Hello, World!");
                        ^
    return 0;
}
Expected one of whitespace, comment, keywords, keywords, (one of {, }, (, ), ;, ,, [, ], ., -> @ create_token), (one of +, -, *, /, %, =, ==, !=, <, >, <=, >=, &&, ||, !, &, |, ^, <<, >> @ create_token), number, string literal, identifier.


In [13]:
tokenizer

Parser(one of whitespace, comment, keywords, keywords, (one of {, }, (, ), ;, ,, [, ], ., -> @ create_token), (one of +, -, *, /, %, =, ==, !=, <, >, <=, >=, &&, ||, !, &, |, ^, <<, >> @ create_token), number, string literal, identifier)

In [11]:
movies = """
12 Monkeys (1995) [1080p] [BluRay] [5.1] [YTS.MX]
1408 (2007) [1080p]
2 Fast 2 Furious (2003) [1080p]
2 Guns (2013) [1080p]
2000.Mules.(2022).1080p.WEB-DL.2CH.H264 - PGPirate.mp4
21 Jump Street (2012) [1080p]
22 Jump Street (2014) [1080p]
28 Days Later (2002)
28 Weeks Later (2007)
30 Minutes or Less (2011)
300 Rise of an Empire (2014) [1080p]
47 Ronin (2013) [1080p]
50 First Dates (2004)
65 (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
A Bad Idea Gone Wrong (2017) [720p] [WEBRip] [YTS.MX]
A Beautiful Mind (2001) [1080p]
A Christmas Story (1983)
A Christmas Story (1983) [1080p] [BluRay] [YTS.MX]
A Christmas Story Christmas (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
A Day To Die (2022) [1080p] [BluRay] [5.1] [YTS.MX]
A Good Person (2023) [720p] [WEBRip] [YTS.MX]
A Haunting In Venice (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
A Man Called Otto (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
A Quiet Place Part II (2020) (1080p WEB-DL x265 HEVC 10bit EAC3 Atmos 5.1 Bandi)
A Very Harold and Kumar Christmas (2011)
A.Quiet.Place.Day.One.2024.1080p.WEBRip.1400MB.DD5.1.x264-GalaxyRG[TGx]
About My Father (2023) [720p] [WEBRip] [YTS.MX]
Accident Man (2018) [1080p] [YTS.AG]
Accident Man Hitmans Holiday (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Adele.One.Night.Only.2021.1080p.WEBRip.x264-RARBG
Air (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
Alien Romulus (2024) [1080p] [WEBRip] [5.1] [YTS.MX]
All That Jazz (1979) [BluRay] [720p] [YTS.AM]
All The Devil's Men (2018) [BluRay] [1080p] [YTS.AM]
All The Little Animals (1998) [1080p] [WEBRip] [YTS.MX]
Am I Racist (2024) [720p] [WEBRip] [YTS.MX]
Ambulance (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
American Hustle (2013) [1080p]
American Psyco (2000) 1080p
American Traitor The Trial Of Axis Sally (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
Amsterdam (2022) [1080p] [BluRay] [5.1] [YTS.MX]
Anger Managment (2003)
Apex.2021.1080p.WEBRip.DD5.1.X.264-EVO
Aquaman And The Lost Kingdom (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Arcadian.2024.720p.AMZN.WEBRip.800MB.x264-GalaxyRG[TGx]
Armor (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Arthur.the.King.2024.720p.AMZN.WEBRip.800MB.x264-GalaxyRG[TGx]
Artificial Intelligence (2001) [1080p]
Ash Dust (2022) [1080p] [WEBRip] [YTS.MX]
Assassin Club (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
Athlete A (2020) [1080p] [WEBRip] [5.1] [YTS.MX]
Athlete A (2020) [720p] [WEBRip] [YTS.MX]
Australia (2008)
Ava (2020) [1080p] [BluRay] [5.1] [YTS.MX]
Avatar The Way Of Water (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Azrael (2024) [720p] [WEBRip] [YTS.MX]
Baby Ruby (2022) [720p] [WEBRip] [YTS.MX]
Babylon (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Bad Times At The El Royale (2018) [BluRay] [1080p] [YTS.AM]
Baghead (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Bandit (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Barbarian (2022) [720p] [WEBRip] [YTS.MX]
Bedtime Stories (2008)
Beetlejuice Beetlejuice (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Being The Ricardos (2021) [720p] [WEBRip] [YTS.MX]
Belfast (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
Beneath Us All (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
Big Daddy (1999)
Birds Of Prey And The Fantabulous Emancipation Of One Harley Quinn (2020) [1080p] [WEBRip] [5.1] [YTS.MX]
Black Adam (2022) [REPACK] [1080p] [BluRay] [5.1] [YTS.MX]
Black Lake (2020) [720p] [WEBRip] [YTS.MX]
Black Mass (2015) [1080p] [YTS.AG]
Black Widow (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
Blackbird (2022) [720p] [WEBRip] [YTS.MX]
Blade Runner (1982) [BluRay] [1080p] [YTS.AM]
Blood (2022) [720p] [WEBRip] [YTS.MX]
Bloodshot (2020) [1080p] [WEBRip] [5.1] [YTS.MX]
Blowback (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Blue Beetle (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Body of Lies (2008) [1080p]
Borat.2.Subsequent.Moviefilm.2020.HDRip.XviD.AC3-EVO[TGx]
Borderlands (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Boss.Level.2020.1080p.WEBRip.x264-RARBG
Boston Strangler (2023) [720p] [WEBRip] [YTS.MX]
Boy.Kills.World.2023.1080p.WEBRip.1400MB.DD5.1.x264-GalaxyRG[TGx]
Breakfast At Tiffany's (1961) [BluRay] [720p] [YTS.AM]
Breathe.2024.1080p.AMZN.WEBRip.DDP5.1.x265.10bit-GalaxyRG265[TGx]
Bridge Of The Doomed (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Brothers (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Bull (2021) [720p] [BluRay] [YTS.MX]
Bullet Train (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Cabaret (1972) [YTS.AG]
Caddo Lake (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Capone (2020) [1080p] [WEBRip] [5.1] [YTS.MX]
Captive State (2019) [BluRay] [1080p] [YTS.AM]
Carnifex (2022) [720p] [WEBRip] [YTS.MX]
Castle.Falls.2021.1080p.WEBRip.x264-RARBG
Cellphone (2023) [720p] [WEBRip] [YTS.MX]
Central Intelligence (2016) [YTS.AG]
Changing Lanes (2002) [720p] [BluRay] [YTS.MX]
Chaos.Walking.2021.720p.WEBRip.800MB.x264-GalaxyRG[TGx]
Charade (1963) [BluRay] [720p] [YTS.AM]
Chef (2014)
Chicago (2002)
Christmas Bloody Christmas (2022) [720p] [WEBRip] [YTS.MX]
Christmas Vacation (1989)
Chupa (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
Cinderella.2021.720p.AMZN.WEBRip.800MB.x264-GalaxyRG[TGx]
Clash of the Titans (2010) [1080p]
Clean (2020) [1080p] [WEBRip] [5.1] [YTS.MX]
Cocaine Bear (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
Code 8 (2019) [WEBRip] [1080p] [YTS.LT]
Code 8 Part II (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Cold Moutians (2003)
Colonials (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
Conclave (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Conjuring The Beyond (2022) [720p] [WEBRip] [YTS.MX]
Contraband (2012) [1080p]
Cosmic Sin (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
Cowboys and Aliens (2011) EXTENDED 1080p BluRay x264 Dual Audio Hindi English AC3 5.1 - MeGUiL
Crimes.of.the.Future.2022.1080p.WEBRip.x265-RARBG
Cruella.2021.1080p.DSNP.WEB-DL.DDP5.1.Atmos.H.264-CMRG[TGx]
Dancer In The Dark (2000) [BluRay] [720p] [YTS.AM]
Dark Harvest (2023) [720p] [WEBRip] [YTS.MX]
Dark Waters (2019) [1080p] [WEBRip] [5.1] [YTS.MX]
Dark.Cloud.2022.1080p.BluRay.x265-RBG.mp4
Day Shift (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Dead For A Dollar (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Deadpool (2016) [1080p] [YTS.AG]
Deadpool 2 (2018) [WEBRip] [1080p] [YTS.AM]
Deadstream (2022) [720p] [WEBRip] [YTS.MX]
Dear.Evan.Hansen.2021.720p.WEBRip.x264.AAC-[YTS.MX].mp4
Death On The Nile (2022) [1080p] [BluRay] [5.1] [YTS.MX]
Death.in.Texas.2021.1080p.WEB-DL.DD5.1.H.264-EVO[TGx]
Deck The Halls (2006) [BluRay] [720p] [YTS.AM]
Delirium (2018) [BluRay] [1080p] [YTS.AM]
Delirium.2018.HDRip.XviD.AC3-EVO
Demonic Christmas Tree (2022) [1080p] [BluRay] [5.1] [YTS.MX]
Despicable.Me.4.2024.1080p.WEBRip.1400MB.DD5.1.x264-GalaxyRG[TGx]
Disenchanted (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Divergent (2014) [1080p]
Doctor Sleep (2019) [720p] [WEBRip] [YTS.LT]
Doctor Strange In The Multiverse Of Madness (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Dog (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Dont Suck (2023) [1080p] [WEBRip] [YTS.MX]
Dont.Breathe.2.2021.720p.WEBRip.800MB.x264-GalaxyRG[TGx]
Downfall (2004) [BluRay] [1080p] [YTS.AM]
Downton Abbey (2019) [WEBRip] [1080p] [YTS.LT]
Downton Abbey A New Era (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Dragon Ball Super Super Hero (2022) [720p] [BluRay] [YTS.MX]
Dream Scenario (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Drive (2011) [1080p] [BluRay] [5.1] [YTS.MX]
Drive Angry (2011) [BluRay] [1080p] [YTS.AM]
Dune (1984) [1080p]
Dune.2021.1080p.HDRip.X264-EVO[TGx]
Dune.Part.Two.2024.1080p.WEBRip.DDP2.0.x265.10bit-GalaxyRG265[TGx]
Dungeons Dragons Honor Among Thieves (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
E.T. The Extra-Terrestrial (1982) [1080p] [YTS.AG]
East Of The Mountains (2021) [720p] [BluRay] [YTS.MX]
Edge of Darkness (2010) [1080p]
Edge of Tomorrow (2014) [1080p]
Elvis (2022) [720p] [WEBRip] [YTS.MX]
Emily The Criminal (2022) [720p] [WEBRip] [YTS.MX]
Enchanted (2007) [BluRay] [1080p] [YTS.AM]
Ender's Game (2013) [1080p]
Enemy (2013) [1080p]
Enola Holmes 2 (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Equilibrium (2002) [1080p] [BluRay] [5.1] [YTS.MX]
Eraser Reborn (2022) [1080p] [WEBRip] [YTS.MX]
Escape Room (2019) [BluRay] [1080p] [YTS.AM]
Escape.Room.Tournament.of.Champions.2021.EXTENDED.1080p.WEBRip.6CH.x265.HEVC-PSA
Eternals (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
Everything Everywhere All At Once (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Evil Dead Rise (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
Expend4bles (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Expired.2022.1080p.WEBRip.x265-RARBG
Extraction (2020) [1080p] [WEBRip] [5.1] [YTS.MX]
Extraction 2 (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
F9 (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
Fall (2022) [720p] [WEBRip] [YTS.MX]
Fantastic Beasts The Secrets Of Dumbledore (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Fast Five (2011) [1080p]
Fast X (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Fast and Furious (2009) [1080p]
Father Stu (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Fatman.2020.1080p.WEBRip.x264-RARBG
Fiddler On The Roof (1971) [YTS.AG]
Fight Club (1999) [REPACK] [1080p] [BluRay] [5.1] [YTS.MX]
Finch (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
Find Her (2022) [1080p] [WEBRip] [YTS.MX]
Five Nights At Freddys (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Ford V Ferrari (2019) [1080p] [WEBRip] [5.1] [YTS.LT]
Forrest.Gump.1994.1080p.BluRay.10bit.HEVC.6CH.MkvCage.ws.mkv
Funny Face (1957) [BluRay] [720p] [YTS.LT]
Furiosa A Mad Max Saga (2024) [1080p] [WEBRip] [5.1] [YTS.MX]
Furious 6 (2013) [1080p]
Furious Seven (2015) [1080p]
Gaia (2021) [REPACK] [720p] [WEBRip] [YTS.MX]
Gangster Squad (2013) [1080p]
Get The Gringo (2012) [1080p]
Ghostbusters (1984) [1080p]
Ghostbusters Afterlife (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
Ghostbusters.Frozen.Empire.2024.1080p.AMZN.WEBRip.1400MB.DD5.1.x264-GalaxyRG[TGx]
Gladiator (2000)
Gladiator II (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Gods Country (2022) [720p] [WEBRip] [YTS.MX]
Godzilla.vs.Kong.2021.1080p.WEBRip.x264-RARBG
Godzilla.x.Kong.The.New.Empire.2024.REPACK2.1080p.WEBRip.1400MB.DD5.1.x264-GalaxyRG[TGx]
Goodfellas (1990)
Gosford Park (2001) [1080p] [YTS.AG]
Green Lantern (2011) [BluRay] [1080p] [YTS.AM]
Green Zone (2010) [1080p]
Greenland (2020) [1080p] [WEBRip] [5.1] [YTS.MX]
Gremlins (1984) [1080p]
Gretel & Hansel (2020) [1080p] [BluRay] [5.1] [YTS.MX]
Groundhog Day (1993)
Groundhog Day (1993) [1080p] [BluRay] [5.1] [YTS.MX]
Grown Ups (2010)
Grown Ups 2 (2013)
Guardians Of The Galaxy Vol. 3 (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Guillermo Del Toros Pinocchio (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Hairspray (2007) [1080p]
Halloween Kills (2021) [720p] [WEBRip] [YTS.MX]
Halo Nightfall (2014) [NORDIC] [1080p] [BluRay] [5.1] [YTS.MX]
Hannibal Rising (2007)
Happy Gilmore (1996)
Harry Potter 20th Anniversary Return To Hogwarts (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Haunted Mansion (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Held (2020) [720p] [WEBRip] [YTS.MX]
Hellboy (2004) [1080p]
Hellboy The Crooked Man (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Hellboy The Golden Army (2008) [1080p]
Hellraiser (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Hercules (1997) [1080p] [YTS.AG]
Heretic (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
High Heat (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Hit.Man.2023.1080p.WEBRip.1400MB.DD5.1.x264-GalaxyRG[TGx]
Hitmain (2007) [1080p]
Hitman.Agent.47.2015.720p.BRRip.x264.AAC-ETRG
Home Sweet Home Alone (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
Hot Fuzz (2007) [1080p]
House Of Gucci (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
I Am Legend AlTERNATE ENDING (2007) [1080p]
IF (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
In The Land Of Saints And Sinners (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
Inception (2010) [1080p]
Indiana Jones And The Dial Of Destiny (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Indiana Jones and the Kingdom of the Crystal Skull (2008) [1080p]
Insomnia (2002) [1080p]
Interview with the Vampire (1994) [1080p]
Into the Woods (2014)
Invitation To A Murder (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
It (2017) [YTS.AG]
It Aint Over (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
It Aint Over (2022) [720p] [WEBRip] [YTS.MX]
It Chapter Two (2019) [WEBRip] [720p] [YTS.LT]
Jack Ryan Shadow Recruit (2014) [1080p]
James Bond The World Is Not Enough (1999) [1080p]
Jarhead (2005) [1080p]
Jingle All the Way EXTENDED (1996)
John And The Hole (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
John Wick (2014) [1080p] [YTS.AG]
John Wick Chapter 2 (2017) [1080p] [YTS.AG]
John Wick Chapter 3 - Parabellum (2019) [BluRay] [1080p] [YTS.LT]
John Wick Chapter 3 - Parabellum (2019) [WEBRip] [1080p] [YTS.LT]
John.Wick.Chapter.4.2023.2160p.WEBRip.3999MB.DDP5.1.x264-GalaxyRG[TGx]
Jojo Rabbit (2019) [1080p] [WEBRip] [5.1] [YTS.MX]
Joker Folie a Deux (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Jujutsu Kaisen 0 The Movie (2021) [JAPANESE] [720p] [BluRay] [YTS.MX]
Jumper (2008) [1080p]
Jungle Cruise (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
Jupiter Ascending (2015) [1080p]
Jurassic World Dominion (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Kandahar (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Killers Of The Flower Moon (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
Kinds Of Kindness (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Kiss.Kiss.Bang.Bang.2005.720p.BluRay.999MB.HQ.x265.10bit-GalaxyRG[TGx]
Knives Out (2019) [1080p] [WEBRip] [5.1] [YTS.MX]
Knock At The Cabin (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
Knowing 2009  (1080p x265 q22 S94 Joy).mkv
Kong Skull Island (2017) [1080p] [YTS.AG]
Kraven The Hunter (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Kronks New Groove (2005) [1080p] [BluRay] [5.1] [YTS.MX]
La La Land (2016) [YTS.AG]
Labyrinth (1986) [1080p] [BluRay] [5.1] [YTS.MX]
Lake George (2024) [720p] [WEBRip] [YTS.MX]
Lamborghini The Man Behind The Legend (2022) [1080p] [BluRay] [5.1] [YTS.MX]
Land Of Bad (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Last Flag Flying (2017) [1080p] [YTS.AG]
Last Night In Soho (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
Last.Looks.2021.720p.WEBRip.800MB.x264-GalaxyRG.mkv
Law.Abiding.Citizen.2009.1080p.BluRay.x264.YIFY.mp4
Legion (2010)
Leo (2023) [720p] [WEBRip] [YTS.MX]
Les Misérables (2012)
Lightyear (2022) [720p] [WEBRip] [YTS.MX]
Little Dixie (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
Little Fockers (2010)
Logan (2017) [1080p] [YTS.AG]
Longlegs (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Lord of War (2005) [1080p]
Lou (2022) [720p] [WEBRip] [YTS.MX]
Love Courage And The Battle Of Bushy Run (2024) [720p] [WEBRip] [YTS.MX]
Love.Lies.Bleeding.2024.720p.WEBRip.800MB.x264-GalaxyRG[TGx]
Luca 2021 FullHD 1080p.H264 Ita Eng AC3 5.1 Multisub realDMDJ.mkv
M3GAN (2022) [720p] [WEBRip] [YTS.MX]
Mad God (2021) [1080p] [WEBRip] [YTS.MX]
Mad Max 2 The Road Warrior (1981)
Mad Max Beyond Thunderdome (1985) [YTS.AG]
Maggie Moore s (2023) [720p] [WEBRip] [YTS.MX]
Malignant (2013) [720p] [WEBRip] [YTS.MX]
Malignant (2021) [720p] [BluRay] [YTS.MX]
Mamma Mia (2008)
Mamma Mia! Here We Go Again (2018) [WEBRip] [720p] [YTS.AM]
Manchester By The Sea (2016) [YTS.AG]
Manifest West (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Margin.Call.2011.720p.BluRay.x264.YIFY.mp4
Margin.Call.2011.720p.BluRay.x264.YIFY.srt
Marley and Me (2008) [1080p]
Mary Poppins (1964) [YTS.AG]
Mary Poppins Returns (2018) [BluRay] [720p] [YTS.AM]
May December (2023) [720p] [WEBRip] [YTS.MX]
Mechanic Resurrection (2016) [1080p] [YTS.AG]
Medieval (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Meet The Fockers (2004)
Meet The Parents (2000)
Megalopolis (2024) [720p] [WEBRip] [YTS.MX]
Memento (2000) [1080p]
Memory (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Mile 22 (2018) [WEBRip] [1080p] [YTS.AM]
Million Dollar Baby (2004)
Minions More 1 (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Minions The Rise Of Gru (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Minority Report (2002) [1080p]
Mission Impossible (1996) [1080p] [YTS.AG]
Mission Impossible - Dead Reckoning Part One (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Mission Impossible - Fallout (2018) [BluRay] [1080p] [YTS.AM]
Mission Impossible Ghost Protocol (2011) [1080p]
Mission Impossible II (2000) [1080p] [YTS.AG]
Mission Impossible III (2006) [1080p] [YTS.AG]
Mission Impossible Rogue Nation (2015) [1080p]
Mona Lisa And The Blood Moon (2021) [1080p] [BluRay] [5.1] [YTS.MX]
Money Monster (2016) [YTS.AG]
Moneyball (2011)
Monkey.Man.2024.720p.AMZN.WEBRip.800MB.x264-GalaxyRG[TGx]
Monolith (2022) [720p] [WEBRip] [YTS.MX]
Monster.Hunter.2020.1080p.AMZN.WEBRip.DDP5.1.x264-NTG
Monsters Of Man (2020) [1080p] [BluRay] [5.1] [YTS.MX]
Monstrous (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Moonfall (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Morbius (2022) [1080p] [WEBRip] [YTS.MX]
Mortal Kombat (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
Mortdecai (2015) [1080p]
Moulin Rouge! (2001)
Mr Nobody (2009)
Mr. Harrigans Phone (2022) [720p] [WEBRip] [YTS.MX]
Mufasa The Lion King (2024) [1080p] [WEBRip] [x265] [10bit] [YTS.MX]
Mulan.2020.720p.DSNP.WEBRip.DDP5.1.x264-NOGRP
Murder At Yellowstone City (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Murder Mystery (2019) [WEBRip] [720p] [YTS.LT]
Murder Mystery 2 (2023) [720p] [BluRay] [YTS.MX]
My Big Fat Greek Wedding 3 (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
My Big Fat Greek Wedding 3 (2023) [720p] [WEBRip] [YTS.MX]
My Fair Lady (1964) [BluRay] [720p] [YTS.AM]
My.Son.Hunter.2022.720p.WEBRip.800MB.x264-GalaxyRG.mkv
Mythica A Quest For Heroes (2014) [1080p] [BluRay] [5.1] [YTS.MX]
Nashville (1975) [720p] [BluRay] [YTS.MX]
National Treasure Book of Secrets (2007) [1080p]
News.of.the.World.2020.1080p.WEBRip.x264-RARBG
Nightfire (2020) [1080p] [WEBRip] [YTS.MX]
Nightmare Alley (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
Nine Days (2020) [1080p] [BluRay] [5.1] [YTS.MX]
No Man of God (2021) (1080p BluRay x265 HEVC 10bit AAC 5.1 Tigole)
No One Lives (2012) [BluRay] [1080p] [YTS.LT]
No.Time.To.Die.2021.1080p.WEBRip.x265-RARBG
Nobody.2021.1080p.WEBRip.1400MB.DD5.1.x264-GalaxyRG[TGx]
Nocebo (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Nocturnal Animals (2016) [YTS.AG]
North Country (2005) [WEBRip] [1080p] [YTS.LT]
Nosferatu (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Now You See Me (2013) [1080p]
Now.You.See.Me.2.2016.1080p.BluRay.6CH.ShAaNiG.mkv
Nutcrackers (2024) [720p] [WEBRip] [YTS.MX]
Obi-Wan Kenobi (2022) [1080p] [WEBRip] [YTS.MX]
Oblivion (2013) [1080p]
Oddity (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Old Henry (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
Old Man (2022) [720p] [WEBRip] [YTS.MX]
One Day As A Lion (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
One Shot (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
Oppenheimer (2023) [1080p] [BluRay] [x265] [10bit] [5.1] [YTS.MX]
Out of the Furnace (2013)
Out of the Furnace 2013 1080p BluRay x264 AAC - Ozlem
Overlord (2018) [WEBRip] [1080p] [YTS.AM]
Paint (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
Palmer (2021) [720p] [WEBRip] [YTS.MX]
Panic Room (2002) [BluRay] [1080p] [YTS.AM]
Paradise Highway (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Percy Vs Goliath (2020) [1080p] [WEBRip] [5.1] [YTS.MX]
Pet Sematary Bloodlines (2023) [720p] [WEBRip] [YTS.MX]
Pig.2021.1080p.WEBRip.x264-RARBG
Pirates Of The Caribbean Dead Men Tell No Tales (2017) [1080p] [YTS.AG]
Pirates of the Caribbean Dead Men Tell No Tales (2017) 720p BrRip x264 - VPPV
Pitch Black (2000) [1080p]
Pixels (2015) [1080p]
Planet of the Apes (2001)
Play Dead (2022) [720p] [WEBRip] [YTS.MX]
Poltergeist (1982) [1080p]
Poltergeist (2015) [1080p]
Predestination (2014) [1080p]
Pride and Prejudice (2005)
Pride and Prejudice (2005) [1080p]
Priscilla (2023) [720p] [WEBRip] [YTS.MX]
Prison Break The Final Break (2009) [1080p] [BluRay] [5.1] [YTS.MX]
Prisoners (2013) [1080p]
Prisoners.of.the.Ghostland.2021.1080p.WEBRip.x264-RARBG
Prizefighter The Life Of Jem Belcher (2022) [720p] [BluRay] [YTS.MX]
Project.Gemini.2022.1080p.BluRay.H264.AAC-RARBG
Public Enemies (2009) [1080p]
Pulp.Fiction.1994.720p.BrRip.x264.YIFY.mp4
Pulp.Fiction.1994.720p.BrRip.x264.YIFY.srt
Puss In Boots (2011) [1080p] [BluRay] [5.1] [YTS.MX]
Puss In Boots The Last Wish (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
R.I.P.D.2.Rise.of.the.Damned.2022.720p.BluRay.800MB.x264-GalaxyRG[TGx]
Ravens Hollow (2022) [1080p] [WEBRip] [YTS.MX]
Ray Donovan The Movie (2022) [1080p] [BluRay] [5.1] [YTS.MX]
Raya.and.the.Last.Dragon.2021.1080p.DSNP.WEBRip.DDP5.1.x264-TOMMY
Real Steel (2011) [1080p]
Rebel Moon - Part One A Child Of Fire (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Rebel.Moon.Part.Two.The.Scargiver.2024.1080p.NF.WEBRip.1400MB.DD5.1.x264-GalaxyRG[TGx]
Red Dawn (1984) [1080p]
Red Dawn (2012)
Red One (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Red.Stone.2021.1080p.WEBRip.x264-RARBG
Redemption (2013) [BluRay] [720p] [YTS.AM]
Rememory (2017) [1080p] [WEBRip] [YTS.MX]
Reminiscence.2021.1080p.WEBRip.x264-RARBG
Renfield (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
Reptile (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Rescue Dawn (2006) [1080p] [BluRay] [5.1] [YTS.MX]
Rescued By Ruby (2022) [720p] [WEBRip] [YTS.MX]
Reservoir Dogs (1992)
Reservoir Dogs (1992) [1080p]
Resident Evil (2002) [1080p]
Resident Evil Afterlife (2010) [1080p]
Resident Evil Apocalypse (2004) [1080p]
Resident Evil Extinction (2007) [1080p]
Resident Evil Retribution (2012) [1080p]
Resident Evil The Final Chapter (2016) [1080p] [YTS.AG]
Resident.Evil.Welcome.to.Raccoon.City.2021.1080p.WEBRip.x264-RARBG
Road.House.2024.1080p.AMZN.WEBRip.1400MB.DD5.1.x264-GalaxyRG[TGx]
Robert The Bruce (2019) [BluRay] [1080p] [YTS.LT]
Rogue Agent (2022) [720p] [BluRay] [YTS.MX]
Roman Holiday (1953) [720p] [WEBRip] [YTS.MX]
Run.Hide.Fight.2020.1080p.WEBRip.x264-RARBG
Runner Runner (2013)
SAS Red Notice (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
SOUTH.PARK.JOINING.THE.PANDERVERSE.2023.720p.WEBRip.400MB.x264-GalaxyRG[TGx]
Sabrina (1954) [720p] [BluRay] [YTS.MX]
Salems Lot (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Samaritan (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Scarface (1983)
Scary Movie (2000) [1080p] [BluRay] [5.1] [YTS.MX]
Scoop (2006) [YTS.AG]
Secret Headquarters (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Section 8 (2022) [720p] [WEBRip] [YTS.MX]
Selfless (2015) [1080p]
Senna (2010) [720p] [BluRay] [YTS.MX]
Serenity (2005) [1080p] [BluRay] [5.1] [YTS.MX]
Seriously Red (2022) [720p] [WEBRip] [YTS.MX]
Settlers (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
Seventh Son (2014) [1080p]
Shadow Dancer (2012) [REPACK] [1080p] [BluRay] [5.1] [YTS.MX]
Shallow Hal (2001)
Shang-Chi.And.The.Legend.Of.The.Ten.Rings.2021.1080p.BluRay.H264.AAC-RARBG
Shazam Fury Of The Gods (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
Shut.In.2022.1080p.WEBRip.x264-RARBG
Shutter Island (2010) [1080p]
Silent Hill Revelation (2012) [1080p]
Silent Hills (2006)
Silent Night (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Singin' In The Rain (1952) [YTS.AG]
Skull (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Skyfall (2012) [1080p]
Sleepless In Seattle (1993)
Slingshot (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Small Things Like These (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Smile (2022) [720p] [WEBRip] [YTS.MX]
Smile.2022.720p.HC.WEBRip.800MB.x264-GalaxyRG[TGx]
Snake Eyes (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
Snowden.2016.HDRip.XviD.AC3-EVO.avi
South Park (1997) Season 26 S26 (1080p HMAX WEB-DL x265 HEVC 10bit AC3 5.1 Silence)
South Park Post COVID (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
South Park The Streaming Wars (2022) [720p] [WEBRip] [YTS.MX]
South Park The Streaming Wars Part 2 (2022) [720p] [WEBRip] [YTS.MX]
South.Park.Not.Suitable.For.Children.2023.1080p.WEBRip.AAC5.1.10bits.x265-Rapta.mkv
South.Park.Post.Covid.The.Return.of.Covid.2021.1080p.AMZN.WEBRip.DD5.1.X.264-EVO
South.Park.The.End.of.Obesity.2024.1080p.10bit.WEBRip.6CH.x265.HEVC-PSA.mkv
Soylent Green (1973) [BluRay] [1080p] [YTS.AM]
Spectre (2015) [1080p] [YTS.AG]
Spider-Man Far From Home (2019) [BluRay] [1080p] [YTS.LT]
Spider-Man No Way Home (2021) [1080p] [BluRay] [5.1] [YTS.MX]
Spiderhead (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Spirited (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Spy Game (2001) [1080p]
Star Trek (2009) [1080p]
Star Trek Beyond (2016) [YTS.AG]
Star.Wars.The.Book.Of.Boba.Fett.Hardcover.Edition.x265.1080p-NumeralJ.mkv
Step Brothers (2008)
Stowaway (2021) [1080p] [WEBRip] [YTS.MX]
Subservience (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Sunray Fallen Soldier (2024) [1080p] [BluRay] [5.1] [YTS.MX]
Super.8.2011.1080p.BluRay.x264.AAC5.1-[YTS.MX] 2.mp4
Sweeney Todd (2007)
Sympathy For The Devil (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
Tarot.2024.1080p.WEBRip.1400MB.DD5.1.x264-GalaxyRG[TGx]
Taxi Driver (1976) [BluRay] [1080p] [YTS.AM]
Terminator 2 (1991) [1080p] [YTS.AG]
Terminator 3 Rise of The Machines (2003) [1080p]
Terminator Genisys (2015) [1080p]
Terminator Salvation DIRECTORS CUT (2009) [1080p]
Terrifier 2 (2022) [720p] [BluRay] [YTS.MX]
Terror On The Prairie (2022) [1080p] [WEBRip] [YTS.MX]
The Accursed (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
The Adam Project (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
The Adjustment Bureau (2011)
The Amityvill Horror (2005) [ 1080p]
The Angry Birds Movie 2 (2019) [WEBRip] [1080p] [YTS.LT]
The Art Of Racing In The Rain (2019) [BluRay] [1080p] [YTS.LT]
The Art Of Self-Defense (2019) [BluRay] [1080p] [YTS.LT]
The Art Of Self-Defense (2019) [BluRay] [720p] [YTS.LT]
The Assassination Of Jesse James By The Coward Robert Ford (2007) [BluRay] [1080p] [YTS.AM]
The Banshees Of Inisherin (2022) [720p] [BluRay] [YTS.MX]
The Batman (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
The Beekeeper (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
The Best Man (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
The Beta Test (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
The Big Short (2015) [YTS.AG]
The Birdcage (1996) [BluRay] [720p] [YTS.AM]
The Boogeyman (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
The Boss Baby Family Business 2021 720p WEBRip x264 700MB - ShortRips.mkv
The Call Of The Wild (2020) [1080p] [WEBRip] [5.1] [YTS.MX]
The Cannonball Run (1981) [BluRay] [720p] [YTS.AM]
The Card Counter (2021) [720p] [BluRay] [YTS.MX]
The Channel (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
The Chronicles Of Riddick (2004) [BluRay] [1080p] [YTS.AM]
The Commuter (2018) [BluRay] [1080p] [YTS.AM]
The Conjuring The Devil Made Me Do It (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
The Contractor (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
The Covenant (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
The Crow (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
The Death Of Stalin (2017) [BluRay] [1080p] [YTS.AM]
The Departed (2006) [1080p]
The Djinn (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
The Dunes (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
The Emperor's New Groove (2000) [1080p]
The Empty Man (2020) [720p] [WEBRip] [YTS.MX]
The Equalizer 2 (2018) [WEBRip] [1080p] [YTS.AM]
The Equalizer 3 (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
The Expendables (2010) [1080p]
The Expendables 2 (2012) [1080p]
The Expendables 3 (2014) [1080p]
The Fast and the Furious (2001) [1080p]
The Fast and the Furious Tokyo Drift (2006) [1080p]
The Fate Of The Furious (2017) [1080p] [YTS.AG]
The Fifth Element 1997 Remastered 1080p BluRay HEVC x265 5.1 BONE.mkv
The Fighter (2010) [1080p]
The Flash (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
The Forever Purge (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
The Game (1997) [1080p]
The Girl In The Spider's Web (2018) [WEBRip] [1080p] [YTS.AM]
The Good Neighbor (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
The Greatest Showman (2017) [BluRay] [720p] [YTS.AM]
The Guardians Of The Galaxy Holiday Special (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
The Hills Have Eyes (2006) [1080p] [YTS.AG]
The Hitchhiker's Guide to the Galaxy (2005) [1080p]
The Hitmans Wifes Bodyguard (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
The Hours (2002)
The Hunger Games The Ballad Of Songbirds Snakes (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
The Hunt (2020) [1080p] [BluRay] [5.1] [YTS.MX]
The Hurt Locker (2008) [1080p] [BluRay] [5.1] [YTS.MX]
The Illusionist (2006)
The Impossible (2012) 720p_BRrip_scOrp_sujaidr
The Incredible Hulk (2008) [1080p]
The Instigators (2024) [720p] [WEBRip] [YTS.MX]
The Invitation (2022) [720p] [WEBRip] [YTS.MX]
The Killer (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
The Killers Game (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
The Kingdom (2007) [1080p]
The Kitchen (2019) [BluRay] [1080p] [YTS.LT]
The Last Detail (1973) [1080p] [YTS.AG]
The Last Duel (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
The Last Heist (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
The Last Kingdom Seven Kings Must Die (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
The Last Manhunt (2022) [1080p] [BluRay] [5.1] [YTS.MX]
The Last Possession (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
The Last Voyage Of The Demeter (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
The Last Voyage Of The Demeter (2023) [AUDIO FIX] [REPACK] [1080p] [WEBRip] [5.1] [YTS.MX]
The Last Witch Hunter (2015) [1080p] [YTS.AG]
The Last Witness (2018) [WEBRip] [1080p] [YTS.AM]
The Locksmith (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
The Lone Ranger (2013) [1080p]
The Losers (2010) [1080p] [BluRay] [5.1] [YTS.MX]
The Lost City (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
The Machine (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
The Man Who Invented Christmas (2017) [BluRay] [720p] [YTS.AM]
The Marvels (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
The Matrix (1999) [1080p]
The Matrix Reloaded (2003) [1080p]
The Matrix Revolutions (2003) [1080p]
The Mechanic (2011) [1080p] [YTS.AG]
The Menu (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
The Minute You Wake Up Dead (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
The Mist (2007)
The Mist (2007) [1080p] [YTS.AG]
The Mole Undercover In North Korea (2020) [720p] [WEBRip] [YTS.MX]
The Money Pit (1986)
The Mummy (2017) [1080p] [YTS.AG]
The Mummy (2017) [YTS.AG]
The Mummy Returns (2001) [1080p]
The Mummy Tomb of the Dragon Emperor (2008) [1080p]
The Nice Guys (2016) [YTS.AG]
The Night House (2020) [1080p] [WEBRip] [5.1] [YTS.MX]
The Northman (2022) [1080p] [BluRay] [5.1] [YTS.MX]
The Notebook (2004) [1080p]
The Nun (2018) [BluRay] [720p] [YTS.AM]
The Nun II (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
The Pale Blue Eye (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
The Phantom of the Opera (2004) [1080p]
The Place Beyond the Pines (2012) [1080p]
The Popes Exorcist (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
The Portable Door (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
The Power Of The Dog (2021) [1080p] [WEBRip] [YTS.MX]
The Price We Pay (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
The Promise (2016) [1080p] [YTS.AG]
The Promised Land (2023) [1080p] [BluRay] [5.1] [YTS.MX]
The Ritual (2017) [YTS.AG]
The Salvation (2014) [1080p]
The Scorpion King (2002) [1080p] [BluRay] [5.1] [YTS.MX]
The Scorpion King 2 Rise Of A Warrior (2008) [1080p] [BluRay] [5.1] [YTS.MX]
The Scorpion King 3 Battle For Redemption (2012) [1080p] [BluRay] [5.1] [YTS.MX]
The Scorpion King 4 Quest For Power (2015) [1080p] [YTS.AG]
The Scorpion King Book Of Souls (2018) [WEBRip] [1080p] [YTS.AM]
The Shift (2023) [1080p] [WEBRip] [x265] [10bit] [YTS.MX]
The Shining (1980)
The Silent Hour (2024) [1080p] [WEBRip] [5.1] [YTS.MX]
The Sixth Secret (2022) [720p] [WEBRip] [YTS.MX]
The Sixth Sense (1999) [1080p]
The Sorcerer's Apprentice (2010)
The Sound of Music (1965)
The Stairs (2021) [720p] [WEBRip] [YTS.MX]
The Suicide Squad (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
The Super Mario Bros. Movie (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
The Terminator (1984) [1080p]
The Tomorrow War (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
The Tourist (2010) [BluRay] [1080p] [YTS.AM]
The Towering Inferno (1974) [BluRay] [720p] [YTS.AM]
The Town EXTENDED (2010) [1080p]
The Tutor (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
The Valet (2022) [720p] [WEBRip] [YTS.MX]
The Vault (2021) [1080p] [BluRay] [5.1] [YTS.MX]
The Visit (2015) [1080p]
The Void (2016) [1080p] [YTS.AG]
The Wall (2017) [1080p] [YTS.AG]
The Water Horse (2007) [720p] [BluRay] [YTS.MX]
The Whale (2022) [720p] [WEBRip] [YTS.MX]
The Witcher S03 PART1 4k to 1080p NF WEBrip x265 DD5.1 D0ct0rLew[SEV]
The Woman In The Window (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
The World Made Straight (2015) [1080p]
The World's End (2013) [1080p]
The.Boy.and.the.Heron.2023.DUBBED.720p.AMZN.WEBRip.800MB.x264-GalaxyRG[TGx]
The.Courier.2020.1080p.WEBRip.DD5.1.x264-CM
The.Dark.Tower.2017.1080p.BluRay.x264-[YTS.AG].mp4
The.Expendables.2010.1080p.BrRip.x264.YIFY.mp4
The.Fall.Guy.2024.1080p.AMZN.WEBRip.1400MB.DD5.1.x264-GalaxyRG[TGx]
The.Gilded.Age.S02E01.1080p.HEVC.x265-MeGusta[TGx]
The.Gilded.Age.S02E05.iNTERNAL.1080p.HEVC.x265-MeGusta[eztv.li].mkv
The.Green.Knight.2021.1080p.WEBRip.x265-RBG
The.Hobbit.Battle.Of.The.Five.Armies.2014.1080p.WEBRip.x264-tomcat12[ETRG]
The.Hyperions.2022.1080p.WEBRip.x265-RARBG
The.King's.Man.2021.1080p.10bit.WEBRip.6CH.x265.HEVC-PSA
The.Marksman.2021.1080p.WEBRip.x264-RARBG
The.Matrix.Resurrections.2021.1080p.10bit.WEBRip.6CH.x265.HEVC-PSA
The.Mechanic.2.Resurrection.2016.1080p.BluRay.x265.10bit-z97
The.Ministry.of.Ungentlemanly.Warfare.2024.1080p.AMZN.WEBRip.1400MB.DD5.1.x264-GalaxyRG[TGx]
The.Misfits.2021.1080p.AMZN.WEBRip.DDP5.1.x264-CM
The.New.Mutants.2020.1080p.BluRay.H264.AAC-RARBG
The.Protege.2021.1080p.AMZN.WEBRip.DDP2.0.x264-LUT
The.Ring.2002.720p.BluRay.999MB.HQ.x265.10bit-GalaxyRG[TGx]
The.Super.Mario.Bros.Movie.2023.1080p.WebRip.X264.Will1869.mp4
The.Unholy.2021.720p.WEBRip.800MB.x264-GalaxyRG[TGx]
The.War.with.Grandpa.2020.720p.WEBRip.800MB.x264-GalaxyRG[TGx]
The.Wolfman.2010.720p
They Live (1988)
This Is Me... Now (2024) [720p] [WEBRip] [YTS.MX]
Thor Love And Thunder (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Those Who Wish Me Dead (2021) [1080p] [WEBRip] [5.1] [YTS.MX]
Tickle Me (1965) [720p] [BluRay] [YTS.MX]
Titanic (1997) [YTS.AG]
To Catch A Killer (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
To End All War Oppenheimer The Atomic Bomb (2023) [720p] [WEBRip] [YTS.MX]
To.Catch.A.Killer.2023.720p.AMZN.WEBRip.800MB.x264-GalaxyRG[TGx]
Tommy (1975) [720p] [BluRay] [YTS.MX]
Tommy Boy (1995) [BluRay] [720p] [YTS.AM]
Top Gun (1986) [1080p]
Top Gun Maverick (2022) [IMAX] [REPACK] [1080p] [WEBRip] [5.1] [YTS.MX]
Transformers Age of Extinction (2014) [1080p]
Transformers Rise Of The Beasts (2023) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Transformers The Last Knight (2017) [1080p] [YTS.AG]
Trap (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Trollhunters.Rise.of.the.Titans.2021.720p.NF.WEBRip.800MB.x264-GalaxyRG[TGx]
Tucker.The.Man.And.His.Dream.1988.720p.BluRay.x264-[YTS.AM].mp4
Turner & Hooch (1989) [1080p] [BluRay] [5.1] [YTS.MX]
Turner & Hooch (1989) [720p] [BluRay] [YTS.MX]
Turning Red (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Twisters (2024) [1080p] [WEBRip] [5.1] [YTS.MX]
Uncharted (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Undefeated (2011) [1080p]
Undergods (2020) [1080p] [WEBRip] [5.1] [YTS.MX]
Underworld Awakening (2012) [1080p]
Underworld Blood Wars (2016) [1080p] [YTS.AG]
Underworld EXTENDED (2003) [1080p]
Underworld Evolution (2006) [1080p]
Underworld Rise of the Lycans (2009) [1080p]
Unhinged (2020) [1080p] [BluRay] [5.1] [YTS.MX]
V H S Beyond (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Venom The Last Dance (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Venom.Let.There.Be.Carnage.2021.2160p.HDR10Plus.WEBRip.6CH.x265.HEVC-PSA
Vesper (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Vicious Fun (2020) [1080p] [BluRay] [YTS.MX]
Violent Night (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
War Dogs (2016) [1080p] [YTS.AG]
War For The Planet Of The Apes (2017) [1080p] [YTS.AG]
War Horse (2011)
War of the Worlds (2005) [1080p]
Warcraft (2016) [1080p] [YTS.AG]
Watcher (2022) [720p] [WEBRip] [YTS.MX]
Waterworld (1995) [1080p]
Werewolf By Night (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
West Side Story (2021) [720p] [BluRay] [YTS.MX]
What Happened To Monday (2017) [BluRay] [1080p] [YTS.AM]
What.is.a.Woman.2022.720p.WEBRip.800MB.x264-GalaxyRG[TGx]
Wicked (2024) [1080p] [WEBRip] [x265] [10bit] [5.1] [YTS.MX]
Willy's.Wonderland.2021.1080p.10bit.BluRay.6CH.x265.HEVC-PSA
Wind.River.2017.1080p.WEB-DL.DD5.1.H264-FGT
Winnie The Pooh Blood And Honey (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
Witch Hunt (2021) [720p] [WEBRip] [YTS.MX]
Wolf Hound (2022) [720p] [WEBRip] [YTS.MX]
Wonder Woman (2017) [1080p] [YTS.AG]
Wonder.Woman.1984.2020.IMAX.1080p.WEB.H264-NOGRP
Wonka (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
World War Z (2013) [1080p]
Wrath of the Titans (2012) [1080p]
Wrath.of.Man.2021.1080p.WEBRip.1400MB.DD5.1.x264-GalaxyRG[TGx]
Your Lucky Day (2023) [1080p] [WEBRip] [5.1] [YTS.MX]
Zack.Snyders.Justice.League.2021.1080p.HDRip.AAC2.0.x264-PD[TGx]
Zero Contact (2022) [1080p] [WEBRip] [5.1] [YTS.MX]
Zero Dark Thirty (2012) [1080p]
Zodiac (2007)
"""

In [29]:
for year in range(1980, 2026):
    movies = movies.replace(f'({year})', '')

In [46]:
movies = movies.replace('x265', '')

In [47]:
movies = movies.replace('[]', '')

In [49]:
movies = '\n'.join(m.strip() for m in movies.split('\n'))

In [50]:
print(movies)


12 Monkeys
1408
2 Fast 2 Furious
2 Guns
2000.Mules...WEB-DL.. -
21 Jump Street
22 Jump Street
28 Days Later
28 Weeks Later
30 Minutes or Less
300 Rise of an Empire
47 Ronin
50 First Dates
65
A Bad Idea Gone Wrong
A Beautiful Mind
A Christmas Story
A Christmas Story
A Christmas Story Christmas
A Day To Die
A Good Person
A Haunting In Venice
A Man Called Otto
A Quiet Place Part II  ( WEB-DL  HEVC  EAC3 Atmos  Bandi)
A Very Harold and Kumar Christmas
A.Quiet.Place.Day.One.2024...1400MB.DD.
About My Father
Accident Man
Accident Man Hitmans Holiday
Adele.One.Night.Only.2021...-
Air
Alien Romulus
All That Jazz (1979)
All The Devil's Men
All The Little Animals
Am I Racist
Ambulance
American Hustle
American Psyco
American Traitor The Trial Of Axis Sally
Amsterdam
Anger Managment
Apex.2021...DD.X.264-EVO
Aquaman And The Lost Kingdom
Arcadian.2024..AMZN..800MB.
Armor
Arthur.the.King.2024..AMZN..800MB.
Artificial Intelligence
Ash Dust
Assassin Club
Athlete A
Athlete A
Australia
Ava
Avatar The Wa